# EDINET XBRL Analysis

設定を入力し、実行ボタンを押すだけで全自動でCSVを作成します。

In [ ]:
%load_ext autoreload
%autoreload 2
import edinet_xbrl_parser as parser
from datetime import datetime
import os

# =============================================================================
# 1. インプット設定
# =============================================================================

# XBRL Zipファイルがあるフォルダ
DIR_XBRL_ZIPS = r"XXX"

# 金融庁タクソノミ Zipファイルがあるフォルダ
DIR_TAXONOMY  = r"XXX"

# 企業ごとのマップJSONを保存するフォルダ (自動生成)
DIR_MAP_CACHE = r"company_maps"

# 抽出対象の企業 (パイプ区切りでOR検索。コード、社名など)
TARGET_COMPANIES = ['AAA']

# 抽出対象の会計年度
TARGET_YEARS = [f"FY{y}" for y in range(2015, 2025)]

# 企業別辞書を強制的に作り直すか？ (通常は False でOK)
REBUILD_MAP = False

# 保存先の明示的指定
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
SAVE_DIR = os.path.join('out', f"result-{timestamp}")

# =============================================================================
# 2. 実行
# =============================================================================
df_result = parser.execute_process(
    xbrl_zip_dir=DIR_XBRL_ZIPS,
    taxonomy_dir=DIR_TAXONOMY,
    map_cache_dir=DIR_MAP_CACHE,
    target_companies=TARGET_COMPANIES,
    target_years=TARGET_YEARS,
    rebuild_map=REBUILD_MAP,
    save_dir=SAVE_DIR
)

In [ ]:
# クラスファイルをインポート
%load_ext autoreload
%autoreload 2
from financial_analyzer import FinancialAnalyzer

# -------------------------------------------------------
# 1. 解析の実行
# -------------------------------------------------------
# df_result は edinet_xbrl_parser.execute_process() の戻り値
analyzer = FinancialAnalyzer(df_result)

# 必要ならここで設定を上書き・追加できる（拡張性）
# 例: analyzer.pl_metrics['売上高']['tags'].append('MyCustomTag')

# PL分析を実行
df_pl = analyzer.analyze_pl()
# BS分析を実行
df_bs = analyzer.analyze_bs()

# -------------------------------------------------------
# 2. 結果のマージ
# -------------------------------------------------------
df_final = analyzer.calculate_efficiency_metrics(df_pl, df_bs)

# -------------------------------------------------------
# 3. CSV保存 (分析結果)
# -------------------------------------------------------
#df_pl.to_csv("df_pl.csv", index=False, encoding='utf-8-sig')
#df_bs.to_csv("df_bs.csv", index=False, encoding='utf-8-sig')

# Slide Generator

既存のdfを適当な形に整形のうえ、ぱわぽに埋め込みます。

In [ ]:
# ==============================================================================
# 1. 初期設定とライブラリのインポート
# ==============================================================================
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import sys
import os
from slides_core import SlideConfig, PowerPointGeneratorEngine

# ==============================================================================
# 2. 環境設定 (Config Configuration)
# ==============================================================================

# 1. テンプレートファイルのパス
template_path = r"XXX"

# 2. 出力先フォルダ設定
output_dir = os.path.abspath(SAVE_DIR)
print(f"今回の出力先フォルダ: {output_dir}")

# 設定オブジェクトの生成
try:
    config = SlideConfig(template_path=template_path, output_dir=output_dir, split_ratio=0.6)
    print("✅ 設定コンフィグの初期化に成功しました。")
except Exception as e:
    print("❌ 設定エラー:", e)
    raise e

# ==============================================================================
# 3. スライド構成とコンテンツの定義 (メイン処理)
# ==============================================================================

# 企業ごとにループ処理
for company_name in df_final['企業名'].drop_duplicates():
    print(f"Processing: {company_name}...")
    
    df_target = df_final[df_final['企業名'] == company_name].copy()
    
    # --- データ前処理 (PL用) ---
    fill_cols = ['売上高', '営業利益', '当期純利益', '事業利益率', '純利益率']
    for c in fill_cols:
        if c in df_target.columns:
            df_target[c] = df_target[c].fillna(0)
    
    if df_target.empty:
        continue

    # ------------------------------------------------------------------
    # BSデータの準備とロジック分岐 (銀行 vs 一般)
    # ------------------------------------------------------------------
    latest_row = df_target.sort_values('会計年度').iloc[-1]
    latest_year = latest_row['会計年度']
    
    bs_mapping = {}
    bs_chart_title = "バランスシート"
    unit_scale = 1e12 
    
    code = latest_row.get('証券コード', 0)
    has_loans = latest_row.get('貸出金', 0) > 0
    has_deposits = latest_row.get('預金', 0) > 0
    is_bank = (code == 8306) or (has_loans and has_deposits)
    
    # 必要な値を df_target に定数として結合するための辞書
    bs_values = {}

    if is_bank:
        # === 銀行業 (Bank) ===
        total_assets = latest_row.get('総資産', 0)
        
        # 資産
        cash = latest_row.get('現金預け金', 0)
        securities = latest_row.get('有価証券', 0)
        loans = latest_row.get('貸出金', 0)
        other_assets = total_assets - (cash + securities + loans)
        
        # 負債・純資産
        deposits = latest_row.get('預金', 0)
        borrowed = latest_row.get('借用金', 0)
        
        # ★修正点: 純資産のフォールバック (純資産がなければ自己資本を使う)
        net_assets = latest_row.get('純資産')
        if pd.isna(net_assets):
            net_assets = latest_row.get('自己資本', 0)
            
        other_liabs = total_assets - (deposits + borrowed + net_assets)

        bs_values = {
            'BS_TotalAssets': total_assets,
            'BS_Cash': cash,
            'BS_Securities': securities,
            'BS_Loans': loans,
            'BS_OtherAssets': other_assets,
            'BS_Deposits': deposits,
            'BS_Borrowed': borrowed,
            'BS_OtherLiabs': other_liabs,
            'BS_NetAssets': net_assets
        }

        bs_mapping = {
            'unit_scale': unit_scale,
            'total_assets_col': 'BS_TotalAssets',
            'left_label': '資産の部',
            'left_stack': [
                {'col': 'BS_Cash',       'name': '現金預け金', 'color_key': 'sky_blue'},
                {'col': 'BS_Securities', 'name': '有価証券',   'color_key': 'lavender'},
                {'col': 'BS_Loans',      'name': '貸出金',     'color_key': 'pale_pink'},
                {'col': 'BS_OtherAssets','name': 'その他',     'color_key': 'ice_blue'},
            ],
            'right_label': '負債・純資産の部',
            'right_stack': [
                {'col': 'BS_Deposits',   'name': '預金',       'color_key': 'cream'},
                {'col': 'BS_Borrowed',   'name': '借用金',     'color_key': 'salmon'},
                {'col': 'BS_OtherLiabs', 'name': 'その他負債', 'color_key': 'mint'},
                {'col': 'BS_NetAssets',  'name': '純資産',     'color_key': 'spring_green'},
            ]
        }
        bs_chart_title = f"{company_name} バランスシート (銀行モデル)"

    else:
        # === 一般事業会社 (General) ===
        total_assets = latest_row.get('総資産', 0)
        bs_values['BS_TotalAssets'] = total_assets
        
        # ★修正点: 純資産のフォールバック
        net_assets = latest_row.get('純資産')
        if pd.isna(net_assets):
            net_assets = latest_row.get('自己資本', 0)
        bs_values['BS_NetAssets'] = net_assets

        # --- 左側 (資産) の構築 ---
        cur_assets = latest_row.get('流動資産')
        left_stack_def = []
        
        # 流動資産が存在する場合のみ分割
        if pd.notna(cur_assets) and cur_assets > 0:
            fix_assets = total_assets - cur_assets
            bs_values['BS_CurAssets'] = cur_assets
            bs_values['BS_FixAssets'] = fix_assets
            
            left_stack_def = [
                {'col': 'BS_CurAssets', 'name': '流動資産', 'color_key': 'sky_blue'},
                {'col': 'BS_FixAssets', 'name': '固定資産', 'color_key': 'cadet_blue'},
            ]
        else:
            # 内訳がない場合は総資産のみ表示
            left_stack_def = [
                {'col': 'BS_TotalAssets', 'name': '総資産', 'color_key': 'cadet_blue'}
            ]

        # --- 右側 (負債・純資産) の構築 ---
        cur_liabs = latest_row.get('流動負債')
        right_stack_def = []
        
        # 流動負債が存在する場合のみ分割
        if pd.notna(cur_liabs) and cur_liabs > 0:
            fix_liabs = total_assets - (cur_liabs + net_assets)
            if fix_liabs < 0: fix_liabs = 0
            
            bs_values['BS_CurLiabs'] = cur_liabs
            bs_values['BS_FixLiabs'] = fix_liabs
            
            right_stack_def = [
                {'col': 'BS_CurLiabs',  'name': '流動負債', 'color_key': 'coral'},
                {'col': 'BS_FixLiabs',  'name': '固定負債', 'color_key': 'mustard'},
                {'col': 'BS_NetAssets', 'name': '純資産',   'color_key': 'spring_green'},
            ]
        else:
            # 内訳がない場合は負債合計を計算して表示
            total_liabs = total_assets - net_assets
            bs_values['BS_TotalLiabs'] = total_liabs
            
            right_stack_def = [
                {'col': 'BS_TotalLiabs', 'name': '負債',     'color_key': 'mustard'},
                {'col': 'BS_NetAssets',  'name': '純資産',   'color_key': 'spring_green'},
            ]

        bs_mapping = {
            'unit_scale': unit_scale,
            'total_assets_col': 'BS_TotalAssets',
            'left_label': '資産の部',
            'left_stack': left_stack_def,
            'right_label': '負債・純資産の部',
            'right_stack': right_stack_def
        }
        bs_chart_title = f"{company_name} バランスシート ({latest_year})"

    # 値をDataFrameに結合
    for col, val in bs_values.items():
        df_target[col] = val

    
    # ------------------------------------------------------------------
    # スライド定義
    # ------------------------------------------------------------------
    cover_content = {
        'main_title': f'【ご提案資料】{company_name}の財務分析',
        'sub_title': 'Financial Analysis Report',
        'date': pd.Timestamp.now().strftime("%Y年%m月%d日")
    }
    
    def get_label(row, col_name, default):
        label_col = f"{col_name}_ラベル"
        if label_col in row.index and pd.notna(row[label_col]):
            return row[label_col]
        return default

    lbl_sales = get_label(latest_row, '売上高', '売上高')
    lbl_op_profit = get_label(latest_row, '営業利益', '営業利益')
    lbl_net_profit = get_label(latest_row, '当期純利益', '当期純利益')

    slides_structure = [
        # --- スライド1 ---
        {
            'slide_title': '業績サマリー (売上・営業利益)',
            'category': 'COMBO_BAR_LINE_2AXIS',
            'data_mapping': {
                'x_col': '会計年度',
                'bar_traces': [
                    {'col': '売上高', 'name': lbl_sales, 'color_key': 'midnight_blue'}, 
                    {'col': '営業利益', 'name': lbl_op_profit, 'color_key': 'gold', 'marker_size': 10, 'line_width': 3},
                ],
                'line_traces': [
                    {'col': '事業利益率', 'name': '営業利益率', 'color_key': 'gray_medium'}
                ]
            },
            'chart_text': {
                'title': '売上高・営業利益および利益率の推移',
                'y1_label': '金額 (円)',
                'y2_label': '利益率 (%)'
            },
            'text_section_title': '■ 業績トレンドの評価',
            'text_blocks': [
                {'title': '売上・利益', 'body': '直近のトレンドを確認します。', 'title_color_key': 'white'},
                {'title': '利益率', 'body': '収益性の変化に着目します。', 'accent_color_key': 'gray_medium', 'title_color_key': 'white'}
            ]
        },
        
        # --- スライド2 ---
        {
            'slide_title': '最終利益の推移',
            'category': 'COMBO_BAR_LINE_2AXIS',
            'data_mapping': {
                'x_col': '会計年度',
                'bar_traces': [
                    {'col': '営業利益', 'name': lbl_op_profit, 'color_key': 'midnight_blue'},
                    {'col': '当期純利益', 'name': lbl_net_profit, 'color_key': 'teal'}
                ],
                'line_traces': [
                    {'col': '純利益率', 'name': '純利益率', 'color_key': 'red'}
                ]
            },
            'chart_text': {
                'title': '営業利益・当期純利益の推移',
                'y1_label': '利益額 (円)',
                'y2_label': '利益率 (%)'
            },
            'text_section_title': '■ 収益性の評価',
            'text_blocks': [
                {'title': 'ボトムライン', 'body': '最終利益の確保状況を確認します。', 'accent_color_key': 'dark_slate', 'title_color_key': 'white'}
            ]
        },
        
        # --- スライド3 ---
        {
            'slide_title': '業績サマリー (上下分割レイアウト)',
            'layout_type': 'vertical',
            'category': 'COMBO_BAR_LINE_2AXIS',
            'data_mapping': {
                'x_col': '会計年度',
                'bar_traces': [
                    {'col': '売上高', 'name': lbl_sales, 'color_key': 'midnight_blue'},
                    {'col': '営業利益', 'name': lbl_op_profit, 'color_key': 'gold', 'marker_size': 10, 'line_width': 3},
                ],
                'line_traces': [
                    {'col': '事業利益率', 'name': '営業利益率', 'color_key': 'gray_medium'}
                ]
            },
            'chart_text': {
                'title': '売上高・営業利益の推移',
                'y1_label': '金額 (円)',
                'y2_label': '%'
            },
            'text_section_title': '■ レイアウトの確認',
            'text_blocks': [
                {'title': '確認1', 'body': 'グラフを大きく見せるレイアウトです。', 'accent_color_key': 'dark_slate', 'title_color_key': 'white'},
                {'title': '確認2', 'body': '下のテキストエリアは横並びになります。', 'accent_color_key': 'dark_slate', 'title_color_key': 'white'}
            ]
        }
    ]
    
    # --- スライド4: BS (条件付き追加) ---
    if bs_mapping:
        slides_structure.append({
            'slide_title': '財政状態の概要 (バランスシート)',
            'category': 'BALANCE_SHEET',
            'layout_type': 'vertical',
            'data_mapping': bs_mapping,
            'chart_text': {
                'title': bs_chart_title,
                'y1_label': '金額 (兆円)'
            },
            'text_section_title': '■ 財務健全性の評価',
            'text_blocks': [
                {
                    'title': '資産・負債構成', 
                    'body': '最新年度の財政状態を可視化しました。',
                    'accent_color_key': 'dark_slate',
                    'title_color_key': 'white'
                },
                {
                    'title': '資本の厚み', 
                    'body': '純資産の割合から財務の安全性を確認します。',
                    'accent_color_key': 'dark_slate',
                    'title_color_key': 'white'
                }
            ]
        })

    # ==============================================================================
    # 5. 実行 (Generate)
    # ==============================================================================
    print(f"🚀 Generating slide for {company_name}...")
    try:
        engine = PowerPointGeneratorEngine(config)
        engine.generate(
            df=df_target,
            cover_content=cover_content,
            slides_structure=slides_structure,
            filename_prefix=f"analysis_{company_name}"
        )
        print(f"🎉 Done!\n")
    except Exception as e:
        print(f"⛔ Error: {e}")
        import traceback
        traceback.print_exc()

print("="*60)
print("All tasks finished.")